In [1]:
import numpy as np
import pandas as pd
from skimage.draw import disk
import scanpy as sc

In [2]:
import cv2

In [ ]:
carpetaData = "../02.Seurat/"
carpetaSave = "../07.PASTE2/"
patient = "19"
adatas = []
ims = []
for i in range(1,5):
    adata_name = f"{carpetaData}Paciente{patient}_merge_{i}.h5ad"
    adata = sc.read_h5ad(adata_name)
    adatas.append(adata)
    im_name = f"{carpetaData}AnnData{patient}/spatial_image_slice{i}.png"
    im = cv2.imread(im_name)
    ims.append(im)

adata = adatas[0]
im = ims[0]
adata

AnnData object with n_obs × n_vars = 4992 × 28463
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'name', 'info', 'percent.butterfly', 'tissue', 'row', 'col', 'imagerow', 'imagecol'
    var: 'x'
    uns: 'spatial'
    obsm: 'spatial'

In [4]:
adata.obsm['spatial']
#adata.obsm['rgb']
adata.obs[['row','col','imagerow','imagecol']]

,row,col,imagerow,imagecol
slice19_1_AAACAACGAATAGTTC-1,16,0,3312,1309
slice19_1_AAACAAGTATCTCCCA-1,102,50,13023,11134
slice19_1_AAACAATCTACTAGCA-1,43,3,6360,1899
slice19_1_AAACACCAATAACTGC-1,19,59,3650,12902
slice19_1_AAACAGAGCGACTCCT-1,94,14,12120,4060
...,...,...,...,...
slice19_1_TTGTTTCACATCCAGG-1,42,58,6247,12706
slice19_1_TTGTTTCATTAGTCTA-1,30,60,4892,13099
slice19_1_TTGTTTCCATACAACT-1,27,45,4554,10151
slice19_1_TTGTTTGTATTACACG-1,41,73,6135,15653


In [6]:
adata.uns['spatial']['slice19_1']


{'images': {'lowres': array([[[219, 229, 253],
          [220, 228, 253],
          [220, 228, 253],
          ...,
          [206, 213, 253],
          [206, 211, 252],
          [207, 212, 253]],
  
         [[218, 228, 252],
          [220, 228, 253],
          [220, 228, 253],
          ...,
          [201, 205, 252],
          [203, 208, 252],
          [206, 211, 253]],
  
         [[219, 227, 252],
          [220, 228, 253],
          [220, 228, 253],
          ...,
          [188, 190, 249],
          [189, 192, 247],
          [183, 189, 243]],
  
         ...,
  
         [[142, 146, 219],
          [187, 190, 245],
          [151, 156, 222],
          ...,
          [216, 225, 252],
          [216, 225, 252],
          [218, 226, 253]],
  
         [[ 97,  97, 181],
          [124, 126, 201],
          [125, 128, 205],
          ...,
          [216, 225, 252],
          [216, 225, 252],
          [216, 225, 252]],
  
         [[ 99,  97, 178],
          [128, 127, 207],
    

In [7]:
float(adata.uns['spatial']['slice19_1']['scalefactors']['tissue_hires_scalef'])
adata.n_obs
adata.obsm['spatial'][1][0]
spot_mask = np.zeros((im.shape[0], im.shape[1]), np.uint8)
spot_mask

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [8]:
adata.obsm['spatial']

array([[ 3312,  1309],
       [13023, 11134],
       [ 6360,  1899],
       ...,
       [ 4554, 10151],
       [ 6135, 15653],
       [ 7264,  2685]])

In [ ]:
for i in 0:len(adatas):
    adata = adatas[i]
    im = ims[i]
    # I am using the lowres image, because is the one that was available 
    spot_diam_fullres = float(adata.uns['spatial']['slice19_1']['scalefactors']['spot_diameter_fullres'])
    scale_lowres = float(adata.uns['spatial']['slice19_1']['scalefactors']['tissue_lowres_scalef'])
    spot_diam_lowres = round(spot_diam_fullres * scale_lowres)
    spot_rad_lowres = round(spot_diam_lowres / 2)

    rgb = []
    for i in range(adata.n_obs):
        x, y = round(adata.obsm['spatial'][i][0] * scale_lowres), round(adata.obsm['spatial'][i][1] * scale_lowres)
        spot_mask = np.zeros((im.shape[0], im.shape[1]), np.uint8)
        cv2.circle(spot_mask, (x, y), radius=int(spot_rad_lowres), color=(255, 255, 255), thickness=-1)
        rgb.append(tuple(int(round(c)) for c in cv2.mean(im, spot_mask)[::-1][1:]))

        #rgb.append(cv2.mean(im, spot_mask)[::-1][1:])
        
    adata.obsm['rgb'] = np.array(rgb)

    adata.write(f'{carpetaSave}Paciente{patient}_merge_{i}.h5ad')


In [ ]:
# --- Datos de entrada ---
# Suponiendo que "image" es un array NumPy de la imagen RGB (600x589x3)
# Suponiendo que "spots_df" es un DataFrame con las coordenadas de los spots
# Contiene columnas: ["row", "col", "imagerow", "imagecol"]

lowres_scalef = float(adata.uns['spatial']['slice19_1']['scalefactors']['tissue_lowres_scalef'])
spot_size_um = 55
spot_size_pxl = spot_size_um/lowres_scalef
spot_radius = spot_size_pxl/2  # Ajusta según el tamaño del spot en píxeles

image = adata.uns['spatial']['slice19_1']['images']['hires']
spots_df = adata.obs[['row','col','imagerow','imagecol']]

In [36]:
spots_df.iterrows()

<generator object DataFrame.iterrows at 0x7f37b03e5cf0>

In [43]:
spot = next(spots_df.iterrows())[1]
x, y = int(spot["imagecol"]), int(spot["imagerow"])
rr, cc = disk((y, x), spot_radius, shape=image.shape[:2])
print(rr, cc, (y,x), spot_radius, image.shape[:2])

[] [] (3312, 1309) 821.104107341895 (600, 589)


In [37]:
# Inicializamos una lista para almacenar los valores RGB promedio por spot
spot_colors = []

# Iteramos sobre cada spot
for idx, spot in spots_df.iterrows():
    x, y = int(spot["imagecol"]), int(spot["imagerow"])  # Coordenadas en la imagen

    # Obtener los píxeles dentro del radio del spot
    rr, cc = disk((y, x), spot_radius, shape=image.shape[:2])

    # Extraer los valores RGB de la imagen en esas coordenadas
    spot_pixels = image[rr, cc]

    # Calcular el promedio de los colores RGB
    mean_rgb = np.mean(spot_pixels, axis=0)  # (R_prom, G_prom, B_prom)
    
    # Guardamos el resultado
    spot_colors.append(mean_rgb)

/home/vgaya/anaconda3/envs/py_env/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/vgaya/anaconda3/envs/py_env/lib/python3.13/site-packages/numpy/_core/_methods.py:139: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [ ]:
# Convertimos a DataFrame
rgb_df = pd.DataFrame(spot_colors, columns=["R", "G", "B"], index=spots_df.index)

# Agregamos los valores RGB al DataFrame de spots
spots_df = pd.concat([spots_df, rgb_df], axis=1)

# Guardamos el resultado
spots_df.to_csv("spots_with_rgb.csv", index=False)

print("Archivo generado: spots_with_rgb.csv")